In [845]:
# 在本节中，我们将介绍如何使用PyTorch更方便地实现线性回归的训练
import torch
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
import random
from d2lzh_pytorch import *

In [846]:
# 3.2.1 生成数据集
# 我们生成与上一节中相同的数据集。其中features是训练数据特征，labels是标签
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

In [847]:
# 3.2.2 读取数据
# PyTorch提供了data包来读取数据。由于data常用作变量名，我们将导入的data模块用Data代替。
# 在每一次迭代中，我们将随机读取包含10个数据样本的小批量
import torch.utils.data as Data

batch_size = 10
# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)
# 随机读取小批量
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)

In [848]:
# 这里data_iter的使用跟上一节中的一样。让我们读取并打印第一个小批量数据样本
for X, y in data_iter:
    print(X, y)
    break

tensor([[ 1.5615, -0.5182],
        [-1.1815, -0.1734],
        [-1.0841, -1.1879],
        [-0.1489,  2.1446],
        [-0.9210, -0.9033],
        [ 2.1355, -1.3205],
        [ 1.9058,  0.8325],
        [-0.1821,  2.3559],
        [-0.2980, -0.6898],
        [-1.5194,  0.2645]]) tensor([ 9.0879,  2.4307,  6.0738, -3.3944,  5.4255, 12.9639,  5.1703, -4.1703,
         5.9521,  0.2583])


In [849]:
# 3.2.3 定义模型
# 首先，导入torch.nn模块。实际上，“nn”是neural networks（神经网络）的缩写。
# 顾名思义，该模块定义了大量神经网络的层。之前我们已经用过了autograd，而nn就是利用autograd来定义模型。
# nn的核心数据结构是Module，它是一个抽象概念，既可以表示神经网络中的某个层（layer），也可以表示一个包含很多层的神经网络。
# 在实际使用中，最常见的做法是继承nn.Module，撰写自己的网络/层。
# 一个nn.Module实例应该包含一些层以及返回输出的前向传播（forward）方法。
# 下面先来看看如何用nn.Module实现一个线性回归模型

class LinearNet(nn.Module):   
    # 定义一个名为 LinearNet 的新类，继承自 PyTorch 的 nn.Module，
    # nn.Module 是 PyTorch 中所有神经网络模块的基类，提供了参数管理、设备迁移等基础功能
    def __init__(self, n_feature):
        # 定义类的初始化方法
        # 接收一个参数 n_feature，表示输入特征的维度（即有多少个特征）

        # 调用父类的初始化方法，确保nn.Module的初始化方法被正确调用，第一个参数告诉Python从LinearNet的父类中查找__init__方法
        super(LinearNet, self).__init__()  

        # 创建一个线性层（全连接层）对象，
        # nn.Linear(n_feature, 1) 表示一个从 n_feature 维到 1 维的线性变换：y = Wx + b
        # 将这个线性层保存为类的属性 self.linear
        # 这里输出维度为 1，因为线性回归预测的是一个标量值
        self.linear = nn.Linear(n_feature, 1)

    # forward 定义前向传播：
    # 在 PyTorch 中，必须实现 forward 方法来定义计算逻辑
    # 接收输入数据 x，返回模型的输出 y
    def forward(self, x):
        # 将输入 x 传入线性层进行计算
        # self.linear(x) 等价于执行 y = Wx + b 的线性变换
        # 结果保存在变量 y 中
        y = self.linear(x)
        return y

# 创建 LinearNet 类的一个实例，命名为 net
# num_inputs 是一个之前定义的变量(值为2)，表示输入特征的数量
net = LinearNet(num_inputs)
print(net) # 使用print可以打印出网络的结构

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


事实上我们还可以用nn.Sequential来更加方便地搭建网络，Sequential是一个有序的容器，网络层将按照在传入Sequential的顺序依次被添加到计算图中。

写法一

net = nn.Sequential(

          nn.Linear(num_inputs, 1)

          # 此处还可以传入其他层

          )

写法二

net = nn.Sequential()

net.add_module('linear', nn.Linear(num_inputs, 1))

#net.add_module ......

写法三

from collections import OrderedDict

net = nn.Sequential(OrderedDict([

            ('linear', nn.Linear(num_inputs, 1))

            # ......

            ]))


print(net)

print(net[0])

In [850]:
# 可以通过net.parameters()来查看模型所有的可学习参数，此函数将返回一个生成器
for param in net.parameters():
    print(param)
# 回顾图3.1中线性回归在神经网络图中的表示。
# 作为一个单层神经网络，线性回归输出层中的神经元和输入层中各个输入完全连接。因此，线性回归的输出层又叫全连接层

Parameter containing:
tensor([[-0.0211,  0.2165]], requires_grad=True)
Parameter containing:
tensor([0.4958], requires_grad=True)


注意：torch.nn仅支持输入一个batch的样本，不支持单个样本输入，如果只有单个样本，可使用input.unsqueeze(0)来添加一维

这里我想问一下，我还没有定义 w1, w2, b 这几个权重和偏差，为什么可以打印出可学习参数的具体数字？

GitHub Copilot：
这是一个很好的问题！

在PyTorch中，当你创建网络层时（如nn.Linear），即使你没有显式定义权重和偏置，PyTorch也会自动为你创建并初始化这些参数。

自动参数初始化机制
当你创建nn.Linear(num_inputs, 1)时，以下过程自动发生：

参数创建：

PyTorch自动创建权重矩阵（形状为[1, num_inputs]）
自动创建偏置向量（形状为[1]）

默认初始化：
权重使用何凯明初始化（Kaiming/He initialization）或均匀分布初始化（取决于PyTorch版本）
偏置通常初始化为零

参数注册：
这些参数自动注册为模型的可学习参数
它们被存储在模型的parameters()集合中

# 访问方式：

# 如果使用Sequential

weight = net[0].weight  # 访问第一层的权重

bias = net[0].bias      # 访问第一层的偏置

# 如果使用自定义LinearNet类

weight = net.linear.weight  # 访问linear层的权重

bias = net.linear.bias      # 访问linear层的偏置

In [851]:
# 3.3.4 初始化模型参数
# 在使用net前，我们需要初始化模型参数，如线性回归模型中的权重和偏差。
# PyTorch在init模块中提供了多种参数初始化方法
# 这里的init是initializer的缩写形式
# 我们通过init.normal_将权重参数每个元素初始化为随机采样于均值为0、标准差为0.01的正态分布。偏差会初始化为零
from torch.nn import init

# normal_使用正态分布（高斯分布）随机数初始化权重参数
init.normal_(net.linear.weight, mean=0, std=0.01)
# constant_将张量的所有元素设置为同一个常数值
init.constant_(net.linear.bias, val=0)  
# 也可以直接修改bias的data: net[0].bias.data.fill_(0), 但通常推荐使用init模块的函数，因为它们提供了更统一的接口

Parameter containing:
tensor([0.], requires_grad=True)

In [852]:
# 3.3.5 定义损失函数
# PyTorch在nn模块中提供了各种损失函数，这些损失函数可看作是一种特殊的层，PyTorch也将这些损失函数实现为nn.Module的子类
# 我们现在使用它提供的均方误差损失作为模型的损失函数
loss = nn.MSELoss()

In [853]:
# 3.3.6 定义优化算法
# 同样，我们也无须自己实现小批量随机梯度下降算法。torch.optim模块提供了很多常用的优化算法比如SGD、Adam和RMSProp等
# 下面我们创建一个用于优化net所有参数的优化器实例，并指定学习率为0.03的小批量随机梯度下降（SGD）为优化算法
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optimizer)

# 多参数组示例：
# optimizer = optim.SGD([
#     {'params': [net.linear.weight]}, # 子网络1的参数使用较大的学习率
#     {'params': [net.linear.bias], 'lr': 0.01} # 子网络2的参数使用较小的学习率
# ], lr=0.03)

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.03
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [854]:
# 我们还可以为不同子网络设置不同的学习率，这在finetune时经常用到。例：
# optimizer =optim.SGD([
#                 # 如果对某个参数不指定学习率，就使用最外层的默认学习率
#                 {'params': net.subnet1.parameters()}, # lr=0.03
#                 {'params': net.subnet2.parameters(), 'lr': 0.01}
#             ], lr=0.03)

有时候我们不想让学习率固定成一个常数，那如何调整学习率呢？主要有两种做法。一种是修改optimizer.param_groups中对应的学习率，另一种是更简单也是较为推荐的做法——新建优化器，由于optimizer十分轻量级，构建开销很小，故而可以构建新的optimizer。但是后者对于使用动量的优化器（如Adam），会丢失动量等状态信息，可能会造成损失函数的收敛出现震荡等情况

In [855]:
# 调整学习率
# 在优化器的 param_groups 中，"group" 与您的样本数量和批次大小无关，它指的是模型参数的分组，而不是数据样本的分组

# for param_group in optimizer.param_groups:
#     param_group['lr'] *= 0.1 # 学习率为之前的0.1倍

In [856]:
# 3.3.7 训练模型
# 在使用Gluon训练模型时，我们通过调用optim实例的step函数来迭代模型参数
# 按照小批量随机梯度下降的定义，我们在step函数中指明批量大小，从而对批量中样本梯度求平均
num_epochs = 20
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)   
        # 功能：模型前向传播，计算预测值
        # 过程：
        # 将特征数据 X 输入模型
        # 调用模型的 forward 方法（隐式调用）
        # net(X) 等价于 net.forward(X)
        # 结果：
        # output 包含模型对当前批次的预测结果
        # 对于线性回归，形状为 [batch_size, 1]

        # 功能：计算损失函数值；y.view(-1, 1): 将标签重塑为列向量，确保标签形状与输出一致
        l = loss(output, y.view(-1, 1))   

        # 梯度清零，等价于net.zero_grad()，区别在于 optimizer.zero_grad() 只清零优化器中的参数梯度
        optimizer.zero_grad() 

        # 功能：执行反向传播，计算梯度
        # 过程：
        # 从损失 l 开始，向后计算计算图中每个参数的梯度
        # 使用自动微分计算链式法则
        l.backward()

        # 功能：根据梯度更新模型参数
        # 过程：
        # 使用预设的优化算法（如SGD）更新参数
        # 参数更新公式：param = param - learning_rate * param.grad / batch_size
        # 结果：
        # 模型参数向着减小损失的方向移动
        # 这一步完成后，模型对当前批次数据的拟合能力提升
        optimizer.step()

    if epoch % 5 == 0:
        for param_group in optimizer.param_groups:
            param_group['lr'] *= 0.7 # 学习率为之前的0.7倍(实际上调整学习率有专门的函数)
    # 实际应用中通常使用更温和的策略，如每几个epoch才衰减一次，或使用更小的衰减系数
    
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 1, loss: 0.000176
epoch 2, loss: 0.000173
epoch 3, loss: 0.000071
epoch 4, loss: 0.000063
epoch 5, loss: 0.000182
epoch 6, loss: 0.000068
epoch 7, loss: 0.000047
epoch 8, loss: 0.000112
epoch 9, loss: 0.000119
epoch 10, loss: 0.000146
epoch 11, loss: 0.000177
epoch 12, loss: 0.000066
epoch 13, loss: 0.000097
epoch 14, loss: 0.000144
epoch 15, loss: 0.000105
epoch 16, loss: 0.000072
epoch 17, loss: 0.000039
epoch 18, loss: 0.000080
epoch 19, loss: 0.000152
epoch 20, loss: 0.000035


In [857]:
# 下面我们分别比较学到的模型参数和真实的模型参数。
# 我们从net获得需要的层，并访问其权重（weight）和偏差（bias）。学到的参数和真实的参数很接近
dense = net.linear
print(true_w, dense.weight)
print(true_b, dense.bias)

[2, -3.4] Parameter containing:
tensor([[ 1.9999, -3.3997]], requires_grad=True)
4.2 Parameter containing:
tensor([4.2001], requires_grad=True)


# 小结
使用PyTorch可以更简洁地实现模型。

torch.utils.data模块提供了有关数据处理的工具，torch.nn模块定义了大量神经网络的层，torch.nn.init模块定义了各种初始化方法，torch.optim模块提供了很多常用的优化算法。